In [1]:
from __future__ import print_function
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd

SERVICE_ACCOUNT_FILE = 'keys.json'
# this json file is the file that was generated by making a service account on GCP
#scope can be understood as the type of service we are interacting with and in our case it is spreadsheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds=None

# here the credentials are defined for the api calling which are no other than the json file above 
# and the scope we are going to use

creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


service = build('sheets', 'v4', credentials=creds)

gsheet_list=[]
rlist=[]

# collecting the sheet id from which the data needs to be compiled
# collecting the data range which needs to be copied
# storing both details in two seperate lists

number_of_gsheet=int(input("How many sheets do you want to compile data from"))

for i in range(0,number_of_gsheet):
    sid=input("Google Sheet ID please: ")
    rangeid=input("Give the range for the above sheet: ")
    gsheet_list.append(sid)
    rlist.append(rangeid)

How many sheets do you want to compile data from2
Google Sheet ID please: 1aE3OPQjWBkBFjsVsRv7DPtSWGs-Frm3JqEee0hLx4m4
Give the range for the above sheet: Sheet1!A1:C6
Google Sheet ID please: 1H4uqo50yUQ2GwRypMNxhwtRHjzvjTX2Ydv5HJaLelGE
Give the range for the above sheet: Sheet1!A1:C6


In [5]:
compiled_data=[]

# Call the Sheets API
sheet = service.spreadsheets()
for (sheetid,rid) in zip(gsheet_list,rlist):
    result = sheet.values().get(spreadsheetId=sheetid,range=rid).execute()
    df = pd.DataFrame(result.get('values', []))
    compiled_data.append(df)

In [21]:
# defining a function to transform this data inot xlsx file
# dflist is the dataframe list
# sheets is the list of the names to which you want to write the data to
# file_name is the name of the excel file

#the following code will add the differeent spread sheets into a singl sheet on xlsx file
# we can add different data frome into different sheets on the same excel file however, for that we 
# need to again set different sheet names to a variable and add that sheet name into the defined function

def gsheet_to_xlsx(dflist,  file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row=0
    for dataframe in dflist:
        
        dataframe.to_excel(writer,startrow=row , startcol=0)   
        row=row+len(dataframe)+2
    writer.save()

# for writing the dataframe into different sheets

# sheets=['a','b','c']

# def gsheet_to_xlsx(dflist, sheet_names, file_name):
#     writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
#     row=0
#     for (dataframe,sheets) in zip(dflist,sheets):
        
#         dataframe.to_excel(writer,sheet_name=sheet_name, startrow=row , startcol=0)   
#         row=row+len(dataframe)+2
#     writer.save()
  

In [22]:
gsheet_to_xlsx(compiled_data,'new_compiled.xlsx')